In [1]:
# importing all libraries needed
import numpy as np
import pandas as pd
import os
import time
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression

In [2]:
#Stanting timming process
start = time.time()

In [3]:
# saving the main directory
main_directory = os.getcwd()

In [4]:
#Rreading the name of the subfolder
subfolder_with_path = [f.path for f in os.scandir(main_directory) if f.is_dir()]

In [5]:
subfolder_name = [f.name for f in os.scandir(main_directory) if f.is_dir()]

In [6]:
###############################################################################################
# Importing information from "samples overview", "intensisties template" and "brain allen map"
###############################################################################################

# Changing directory to the "zzz_input_and_reference_files" folder
os.chdir(subfolder_with_path[-1])


# Reading "sample_overview.csv" file
samples_overview = pd.read_csv("sample_overview.csv", 
                               dtype={"MOUSE": "string","SAMPLE": int,
                                      "FILES_HD": int,"FILES_BOX": int,
                                      "hemisphere": "string","STAIN": "string",
                                      "TX_GROUP": "string", 
                                      "GENDER":"string", 
                                      "CONTEXT":"string"})

# Reading "intensities_template.csv" file
intensities_template = pd.read_csv("intensities_template.csv",
                                   dtype={"IDPath": "string", 
                                          "LabelID": int, 
                                          "raw": "string", 
                                          "labelabrv": "string"})
# Reading "brain_allen_map.csv" file
brain_allen_map = pd.read_csv("brain_allen_map.csv", 
                               dtype={"raw": "string","allen_1": "string",
                                      "allen_2": "string","allen_3": "string",
                                      "allen_4": "string","allen_5": "string",
                                      "allen_6": "string","allen_7": "string",
                                      "fine": "string","medium": "string",
                                      "coarse": "string", "all":"string"})


In [7]:
# Changing directory to the "02Volume_files" folder
os.chdir(subfolder_with_path[2])

#Counting number of files on "02Volume_files" folder
list = os.listdir(subfolder_with_path[2]) 
number_files = len(list)
number_volume_files = number_files

In [8]:
#Using and timing  a loop to process all files and populate the main data-frame

volume_data = [] #This array will contain df as elements

i = 0
while i < number_files:

    ##############################################
    # Processing each file
    ##############################################
    
    file = list[i]
    # Saving sample's number
    filename_segmented = file.split('_')
    sample = filename_segmented[1]


    # Reading file's data
    header = ["PIXEL_COUNT"]
    pixels_count_volume = pd.read_csv(file, names = header)
    pixels_count_volume.insert(0, column = 'INTENSITY', value = range(0, len(pixels_count_volume))) 

    # Adding activity columns to intensitites template data-frame
    output = intensities_template.merge(pixels_count_volume, how='left', left_on='LabelID', right_on='INTENSITY')
    output = output.drop(columns=['INTENSITY'])
    volume_label_name = 'VOLUME'
    output = output.rename(columns = {'PIXEL_COUNT':volume_label_name})
    output['VOLUME'] = output['VOLUME'].div(1000000).round(6)
    
    # Checking to which hemisphere the sample belongs to and erasing activity that belong to the other hemisphere
    hemisphere = samples_overview['hemisphere'].loc[samples_overview['SAMPLE'] == int(sample)]
    hemisphere = hemisphere.array
    if hemisphere[0] == 'left':
        output = output[output['LabelID'] > 20000]
    else:
        output = output[output['LabelID'] < 20000]
   

    # Adding a column of the sample number
    sample_column = np.empty(len(output))
    sample = int(sample)
    sample_column.fill(sample)
    output['SAMPLE'] = sample_column
    
    # Erasing the hemisphere label form the "raw" and the and "labelabrv" columns
    output["raw"] = output["raw"].str.replace('left_','')
    output["raw"] = output["raw"].str.replace('right_','')
    output["labelabrv"] = output["labelabrv"].str.replace('L','',1)
    output["labelabrv"] = output["labelabrv"].str.replace('R','',1)
    
    ##########################################################################################################################
    
    # store DataFrame in list
    volume_data.append(output)
    
    i += 1
    
#Joinning all elements of the array in a dataframe
volume_data = pd.concat(volume_data)

In [9]:
volume_data

,IDPath,LabelID,raw,labelabrv,VOLUME,SAMPLE
0,/997/8/567/623/477/278/23/,20023,anterior_amygdalar_area,AAA,0.142518,10.0
1,/997/8/567/688/695/315/31/39/935/,20935,anterior_cingulate_area_dorsal_part_layer_1,ACAd1,0.111832,10.0
2,/997/8/567/688/695/315/31/39/211/,20211,anterior_cingulate_area_dorsal_part_layer_2/3,ACAd2/3,0.162399,10.0
3,/997/8/567/688/695/315/31/39/1015/,21015,anterior_cingulate_area_dorsal_part_layer_5,ACAd5,0.141950,10.0
4,/997/8/567/688/695/315/31/39/919/,20919,anterior_cingulate_area_dorsal_part_layer_6a,ACAd6a,0.072233,10.0
...,...,...,...,...,...,...
1106,/997/1009/967/933/413/,413,vestibular_nerve,vVIIIn,0.054016,9.0
1107,/997/8/343/1065/354/370/765/,765,nucleus_x,x,0.013907,9.0
1108,/997/8/343/1065/354/370/773/,773,hypoglossal_nucleus,XII,0.084287,9.0
1109,/997/8/343/1065/354/370/781/,781,nucleus_y,y,0.005363,9.0


In [10]:
# Changing directory to the "03Volume_in_tissue_files" folder
os.chdir(subfolder_with_path[3])

#Counting number of files on "03Volume_in_tissue_files" folder
list = os.listdir(subfolder_with_path[3]) 
number_files = len(list)
number_volintissue_files = number_files

In [11]:
#Using and timing  a loop to process all files and populate the main data-frame

volume_intissue_data = [] #This array will contain df as elements

#Staring counters

i = 0 #General loop counter

sal_tdt = 0 # Counter of samples where Saline was used and tdTomato expressed
mdma_tdt = 0 # Counter of samples where MDMA was used and tdTomato expressed
meth_tdt = 0 # Counter of samples where Meth was used and tdTomato expressed

sal_cfos = 0 # Counter of samples where Saline was used and cfos expressed
mdma_cfos = 0 # Counter of samples where MDMA was used and cfos expressed
meth_cfos = 0 # Counter of samples where Meth was used and cfos expressed

#These array will contain sample's names as elements
samples_sal_tdt = [] 
samples_mdma_tdt = [] 
samples_meth_tdt = [] 
samples_sal_cfos = []
samples_mdma_cfos = [] 
samples_meth_cfos = []

    ##############################################
    # Processing each file
    ##############################################

while i < number_files:


    file = list[i]
    # Saving sample's number 
    filename_segmented = file.split('_')
    sample = filename_segmented[1]

    # Reading file's data
    header = ["PIXEL_COUNT"]
    pixels_count_volume = pd.read_csv(file, names = header)
    pixels_count_volume.insert(0, column = 'INTENSITY', value = range(0, len(pixels_count_volume))) 

    # Adding activity columns to intensitites template data-frame
    output = intensities_template.merge(pixels_count_volume, how='left', left_on='LabelID', right_on='INTENSITY')
    output = output.drop(columns=['INTENSITY'])
    volume_label_name = 'VOLUME_IN_TISSUE'
    output = output.rename(columns = {'PIXEL_COUNT':volume_label_name})
    output['VOLUME_IN_TISSUE'] = output['VOLUME_IN_TISSUE'].div(1000000).round(6)
    
    # Checking to which hemisphere the sample belongs to and erasing activity that belong to the other hemisphere
    hemisphere = samples_overview['hemisphere'].loc[samples_overview['SAMPLE'] == int(sample)]
    hemisphere = hemisphere.array
    if hemisphere[0] == 'left':
        output = output[output['LabelID'] > 20000]
    else:
        output = output[output['LabelID'] < 20000]

    # Adding a column of the sample number
    sample_column = np.empty(len(output))
    sample = int(sample)
    sample_column.fill(sample)
    output['SAMPLE'] = sample_column
#     output['SAMPLE'] = sample
    
    # Merging activity and sample overview data-frames
    output= pd.merge(output,samples_overview[['SAMPLE','GENDER',
                                             'hemisphere','marker',
                                             'TX_GROUP','CONTEXT']],on='SAMPLE',how='inner')
    
    #Adding a column with the sample's nickname

    if (output['TX_GROUP'].iloc[0] == 'Saline') and (output['marker'].iloc[0] == 'tdtomato'):
        if output['SAMPLE'].iloc[0] not in samples_sal_tdt:
            sal_tdt += 1
        samples_sal_tdt.append(output['SAMPLE'].iloc[0])
        nickname = 'vSAL_' + str(sal_tdt)
        
    elif (output['TX_GROUP'].iloc[0] == 'MDMA') and (output['marker'].iloc[0] == 'tdtomato'):
        if output['SAMPLE'].iloc[0] not in samples_mdma_tdt:
            mdma_tdt += 1 
        samples_mdma_tdt.append(output['SAMPLE'].iloc[0])
        nickname = 'vMDMA_' + str(mdma_tdt)
        
    elif (output['TX_GROUP'].iloc[0] == 'METH') and (output['marker'].iloc[0] == 'tdtomato'):
        if output['SAMPLE'].iloc[0] not in samples_meth_tdt:
            meth_tdt += 1 
        samples_meth_tdt.append(output['SAMPLE'].iloc[0])
        nickname = 'vMETH_' + str(meth_tdt)
        
    elif (output['TX_GROUP'].iloc[0] == 'Saline') and (output['marker'].iloc[0] == 'cfos'):
        if output['SAMPLE'].iloc[0] not in samples_sal_cfos:
            sal_cfos += 1 
        samples_sal_cfos.append(output['SAMPLE'].iloc[0])
        nickname = 'vSAL_' + str(sal_cfos)
        
    elif (output['TX_GROUP'].iloc[0] == 'MDMA') and (output['marker'].iloc[0] == 'cfos'):
        if output['SAMPLE'].iloc[0] not in samples_mdma_cfos:
            mdma_cfos += 1 
        samples_mdma_cfos.append(output['SAMPLE'].iloc[0])
        nickname = 'vMDMA_' + str(mdma_cfos)
        
    elif (output['TX_GROUP'].iloc[0] == 'METH') and (output['marker'].iloc[0] == 'cfos'):
        if output['SAMPLE'].iloc[0] not in samples_meth_cfos:
            meth_cfos += 1 
        samples_meth_cfos.append(output['SAMPLE'].iloc[0])
        nickname = 'vMETH_' + str(meth_cfos)

    nickname_column = np.empty(len(output))    
    output['nickname'] = nickname_column
    output['nickname'] = nickname
    
    # Erasing the hemisphere label form the "raw" and the and "labelabrv" columns
    output["raw"] = output["raw"].str.replace('left_','')
    output["raw"] = output["raw"].str.replace('right_','')
    output["labelabrv"] = output["labelabrv"].str.replace('L','',1)
    output["labelabrv"] = output["labelabrv"].str.replace('R','',1)
    
    ##########################################################################################################################
    
    # store DataFrame in list
    volume_intissue_data.append(output)
    
    i += 1
    
#Joinning all elements of the array in a dataframeo
volume_intissue_data = pd.concat(volume_intissue_data)

In [12]:

volume_intissue_data

,IDPath,LabelID,raw,labelabrv,VOLUME_IN_TISSUE,SAMPLE,GENDER,hemisphere,marker,TX_GROUP,CONTEXT,nickname
0,/997/8/567/623/477/278/23/,20023,anterior_amygdalar_area,AAA,0.141054,10.0,F,left,cfos,METH,group housed,vMETH_1
1,/997/8/567/688/695/315/31/39/935/,20935,anterior_cingulate_area_dorsal_part_layer_1,ACAd1,0.111117,10.0,F,left,cfos,METH,group housed,vMETH_1
2,/997/8/567/688/695/315/31/39/211/,20211,anterior_cingulate_area_dorsal_part_layer_2/3,ACAd2/3,0.162399,10.0,F,left,cfos,METH,group housed,vMETH_1
3,/997/8/567/688/695/315/31/39/1015/,21015,anterior_cingulate_area_dorsal_part_layer_5,ACAd5,0.141950,10.0,F,left,cfos,METH,group housed,vMETH_1
4,/997/8/567/688/695/315/31/39/919/,20919,anterior_cingulate_area_dorsal_part_layer_6a,ACAd6a,0.072233,10.0,F,left,cfos,METH,group housed,vMETH_1
...,...,...,...,...,...,...,...,...,...,...,...,...
551,/997/1009/967/933/413/,413,vestibular_nerve,vVIIIn,0.052276,9.0,F,right,tdtomato,METH,single housed,vMETH_4
552,/997/8/343/1065/354/370/765/,765,nucleus_x,x,0.013907,9.0,F,right,tdtomato,METH,single housed,vMETH_4
553,/997/8/343/1065/354/370/773/,773,hypoglossal_nucleus,XII,0.030265,9.0,F,right,tdtomato,METH,single housed,vMETH_4
554,/997/8/343/1065/354/370/781/,781,nucleus_y,y,0.005324,9.0,F,right,tdtomato,METH,single housed,vMETH_4


In [13]:
#calculating volume ratio
volumes_data = pd.merge(volume_intissue_data,volume_data[['raw','VOLUME','SAMPLE']],on=('raw','SAMPLE'),how='left')
volumes_data['volumes_ratio'] = volumes_data['VOLUME_IN_TISSUE'] / volumes_data['VOLUME'] 



In [14]:
#Pivoting dataframe to have nicknames as columns
pivoted_volumes_data = pd.pivot_table(volumes_data, values='VOLUME_IN_TISSUE', 
                                       index=['raw'],columns=['nickname'], aggfunc=np.sum)

nickname,vMDMA_1,vMDMA_2,vMDMA_3,vMDMA_4,vMDMA_5,vMDMA_6,vMDMA_7,vMETH_1,vMETH_2,vMETH_3,vMETH_4,vSAL_1,vSAL_2,vSAL_3,vSAL_4,vSAL_5,vSAL_6
raw,,,,,,,,,,,,,,,,,
abducens_nucleus,0.007114,0.009127,0.009190,0.008820,0.007697,0.008675,0.009145,0.009687,0.006737,0.007926,0.006983,0.009828,0.007175,0.005554,0.005969,0.009240,0.005177
accessory_facial_motor_nucleus,0.002199,0.002592,0.002138,0.002021,0.002131,0.001770,0.002241,0.002634,0.001331,0.001230,0.002282,0.002624,0.002103,0.002117,0.002248,0.002468,0.001400
accessory_olfactory_bulb_glomerular_layer,0.057630,0.030366,0.022944,0.026680,0.040068,0.003677,0.010502,0.017265,0.027269,0.000000,0.046349,0.000000,0.091065,0.008009,0.021116,0.012829,0.000000
accessory_olfactory_bulb_granular_layer,0.047999,0.034812,0.001071,0.002184,0.021297,0.000000,0.000489,0.000000,0.001805,0.000000,0.018442,0.000000,0.030456,0.000000,0.004804,0.000000,0.000000
accessory_olfactory_bulb_mitral_layer,0.095751,0.075801,0.017360,0.019291,0.080649,0.001910,0.004571,0.001258,0.018205,0.000000,0.057525,0.000000,0.115314,0.000361,0.020812,0.008979,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
visceral_area_layer_5,0.208291,0.236875,0.244129,0.211034,0.196298,0.235516,0.234331,0.212116,0.236790,0.125347,0.233886,0.216283,0.224477,0.227442,0.212117,0.237417,0.134202
visceral_area_layer_6a,0.226138,0.229020,0.248743,0.204469,0.210164,0.223298,0.237222,0.210459,0.241460,0.148187,0.229771,0.230693,0.223378,0.231349,0.251171,0.230213,0.131295
visceral_area_layer_6b,0.019237,0.019174,0.020876,0.017043,0.018265,0.018432,0.020281,0.017806,0.020665,0.015941,0.019280,0.019916,0.019079,0.019339,0.020788,0.019488,0.011250


In [25]:
# Changing directory to the "01Activity_files" folder
os.chdir(subfolder_with_path[1])

#Counting number of files on "01Activity_files" folder
list = os.listdir(subfolder_with_path[1]) 
number_files = len(list)
number_activity_files = number_files

In [41]:
#Using and timing  a loop to process all files and populate the main data-frame

activity_data = [] #This array will contain df as elements

#Staring counters

i = 0 #General loop counter

sal_tdt = 0 # Counter of samples where Saline was used and tdTomato expressed
mdma_tdt = 0 # Counter of samples where MDMA was used and tdTomato expressed
meth_tdt = 0 # Counter of samples where Meth was used and tdTomato expressed

sal_cfos = 0 # Counter of samples where Saline was used and cfos expressed
mdma_cfos = 0 # Counter of samples where MDMA was used and cfos expressed
meth_cfos = 0 # Counter of samples where Meth was used and cfos expressed

#These array will contain sample's names as elements
samples_sal_tdt = [] 
samples_mdma_tdt = [] 
samples_meth_tdt = [] 
samples_sal_cfos = []
samples_mdma_cfos = [] 
samples_meth_cfos = []


    ##############################################
    # Innitiates loop to process each file
    ##############################################


while i < number_files:

    # Working with each file at a time
    file = list[i]
    
    # Saving sample's number
    filename_segmented = file.split('_')
    sample = filename_segmented[1]

    # Reading file's data
    regions_count_activity_fracc = pd.read_csv(file,low_memory=False)
    regions_count_activity_fracc = regions_count_activity_fracc[regions_count_activity_fracc.PIXEL_COUNT != "PIXEL_COUNT"]        

    # Adding-up regions' fracctions
    regions_fracc_1 = regions_count_activity_fracc[["INTENSITY_1", "INTENSITY_1_PERC"]]
    regions_fracc_2 = regions_count_activity_fracc[["INTENSITY_2", "INTENSITY_2_PERC"]]
    regions_fracc_3 = regions_count_activity_fracc[["INTENSITY_3", "INTENSITY_3_PERC"]]
    regions_fracc_1 = regions_fracc_1.rename(columns={"INTENSITY_1": "INTENSITY","INTENSITY_1_PERC" : "COUNTS"})
    regions_fracc_2 = regions_fracc_2.rename(columns={"INTENSITY_2": "INTENSITY","INTENSITY_2_PERC" : "COUNTS"})
    regions_fracc_3 = regions_fracc_3.rename(columns={"INTENSITY_3": "INTENSITY","INTENSITY_3_PERC" : "COUNTS"})
    total_region_activity = regions_fracc_1.append(regions_fracc_2, 
                                                   ignore_index=True).append(regions_fracc_3, ignore_index=True)
    total_region_activity["COUNTS"] = total_region_activity["COUNTS"].astype(float)
    total_region_activity = total_region_activity.groupby(['INTENSITY']).agg('sum').reset_index()
#     total_region_activity["COUNTS"] = total_region_activity["COUNTS"].round(0)
    total_region_activity["INTENSITY"] = total_region_activity["INTENSITY"].astype(int)
        
    # Adding activity columns to intensitites template data-frame
    output = intensities_template.merge(total_region_activity, how='left', left_on='LabelID', right_on='INTENSITY')
    output = output.drop(columns=['INTENSITY'])
    activity_label_name = 'ACTIVITY'
    output = output.rename(columns = {'COUNTS':activity_label_name})
    
    # Checking to which hemisphere the sample belongs to and erasing activity that belong to the other hemisphere
    hemisphere = samples_overview['hemisphere'].loc[samples_overview['SAMPLE'] == int(sample)]
    hemisphere = hemisphere.array
    if hemisphere[0] == 'left':
        output = output[output['LabelID'] > 20000]
    else:
        output = output[output['LabelID'] < 20000]

    # Adding a column of the sample number
    sample_column = np.empty(len(output))
    sample = int(sample)
    sample_column.fill(sample)
    output['SAMPLE'] = sample_column
#     output['SAMPLE'] = sample
    
#     # Adding a column of the rater's name
#     rater_column = np.empty(len(output))
#     output['rater'] = rater_column
#     output['rater'] = rater
    
    # Merging activity and sample overview data-frames
    output= pd.merge(output,samples_overview[['SAMPLE','GENDER',
                                             'hemisphere','marker',
                                             'TX_GROUP','CONTEXT']],on='SAMPLE',how='inner')
    
    
    #Adding a column with the sample's nickname

    if (output['TX_GROUP'].iloc[0] == 'Saline') and (output['marker'].iloc[0] == 'tdtomato'):
        if output['SAMPLE'].iloc[0] not in samples_sal_tdt:
            sal_tdt += 1
        samples_sal_tdt.append(output['SAMPLE'].iloc[0])
        nickname = 'SAL_' + str(sal_tdt)
        
    elif (output['TX_GROUP'].iloc[0] == 'MDMA') and (output['marker'].iloc[0] == 'tdtomato'):
        if output['SAMPLE'].iloc[0] not in samples_mdma_tdt:
            mdma_tdt += 1 
        samples_mdma_tdt.append(output['SAMPLE'].iloc[0])
        nickname = 'MDMA_' + str(mdma_tdt)
        
    elif (output['TX_GROUP'].iloc[0] == 'METH') and (output['marker'].iloc[0] == 'tdtomato'):
        if output['SAMPLE'].iloc[0] not in samples_meth_tdt:
            meth_tdt += 1 
        samples_meth_tdt.append(output['SAMPLE'].iloc[0])
        nickname = 'METH_' + str(meth_tdt)
        
    elif (output['TX_GROUP'].iloc[0] == 'Saline') and (output['marker'].iloc[0] == 'cfos'):
        if output['SAMPLE'].iloc[0] not in samples_sal_cfos:
            sal_cfos += 1 
        samples_sal_cfos.append(output['SAMPLE'].iloc[0])
        nickname = 'SAL_' + str(sal_cfos)
        
    elif (output['TX_GROUP'].iloc[0] == 'MDMA') and (output['marker'].iloc[0] == 'cfos'):
        if output['SAMPLE'].iloc[0] not in samples_mdma_cfos:
            mdma_cfos += 1 
        samples_mdma_cfos.append(output['SAMPLE'].iloc[0])
        nickname = 'MDMA_' + str(mdma_cfos)
        
    elif (output['TX_GROUP'].iloc[0] == 'METH') and (output['marker'].iloc[0] == 'cfos'):
        if output['SAMPLE'].iloc[0] not in samples_meth_cfos:
            meth_cfos += 1 
        samples_meth_cfos.append(output['SAMPLE'].iloc[0])
        nickname = 'METH_' + str(meth_cfos)

    nickname_column = np.empty(len(output))    
    output['nickname'] = nickname_column
    output['nickname'] = nickname

    # Erasing the hemisphere label form the "raw" and the and "labelabrv" columns
    output["raw"] = output["raw"].str.replace('left_','')
    output["raw"] = output["raw"].str.replace('right_','')
    output["labelabrv"] = output["labelabrv"].str.replace('L','',1)
    output["labelabrv"] = output["labelabrv"].str.replace('R','',1)
    
    ##########################################################################################################################
    
    # store DataFrame in list
    activity_data.append(output)
    
    i += 1
    
#Joinning all elements of the array in a dataframe
activity_data = pd.concat(activity_data)

In [42]:
#Replacing empty cells with zeros
activity_data['ACTIVITY'].fillna(0, inplace=True)

In [43]:
# from decimal import localcontext, Decimal, ROUND_HALF_UP
# with localcontext() as ctx: ctx.rounding = ROUND_HALF_UP
# data = {'values': [5.52132, 6.572935, 7.21, 8.755, 9.9989]}
# df = pd.DataFrame(data, columns = ['values'])
# # df['values'].apply(np.ceil)
# Decimal(df['values'])

In [44]:

activity_data.to_csv('activity_data.csv', index=False)

In [19]:
#Merging activity and volume ratio to filter activity where the volume ratio is below 50%
filtered_activity_data = pd.merge(activity_data,
                                  volumes_data[['raw','SAMPLE','volumes_ratio']],
                                  on=['raw','SAMPLE'],
                                  how='left')
filtered_activity_data.loc[(filtered_activity_data['volumes_ratio'] <= 0.5) | (filtered_activity_data['volumes_ratio'] > 1.0)
                           ,'ACTIVITY'] = ''

In [20]:
filtered_activity_data

,IDPath,LabelID,raw,labelabrv,ACTIVITY,SAMPLE,GENDER,hemisphere,marker,TX_GROUP,CONTEXT,nickname,volumes_ratio
0,/997/8/567/623/477/278/23/,20023,anterior_amygdalar_area,AAA,2142.0,2.0,F,left,cfos,Saline,group housed,SAL_1,0.999994
1,/997/8/567/688/695/315/31/39/935/,20935,anterior_cingulate_area_dorsal_part_layer_1,ACAd1,1251.0,2.0,F,left,cfos,Saline,group housed,SAL_1,0.998989
2,/997/8/567/688/695/315/31/39/211/,20211,anterior_cingulate_area_dorsal_part_layer_2/3,ACAd2/3,1941.0,2.0,F,left,cfos,Saline,group housed,SAL_1,1.000000
3,/997/8/567/688/695/315/31/39/1015/,21015,anterior_cingulate_area_dorsal_part_layer_5,ACAd5,1519.0,2.0,F,left,cfos,Saline,group housed,SAL_1,1.000000
4,/997/8/567/688/695/315/31/39/919/,20919,anterior_cingulate_area_dorsal_part_layer_6a,ACAd6a,605.0,2.0,F,left,cfos,Saline,group housed,SAL_1,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
550,/997/1009/967/933/413/,20413,vestibular_nerve,vVIIIn,140.0,2.0,F,left,cfos,Saline,group housed,SAL_1,0.997812
551,/997/8/343/1065/354/370/765/,20765,nucleus_x,x,7.0,2.0,F,left,cfos,Saline,group housed,SAL_1,1.000000
552,/997/8/343/1065/354/370/773/,20773,hypoglossal_nucleus,XII,,2.0,F,left,cfos,Saline,group housed,SAL_1,0.273756
553,/997/8/343/1065/354/370/781/,20781,nucleus_y,y,11.0,2.0,F,left,cfos,Saline,group housed,SAL_1,1.000000


In [21]:
#Pivoting dataframe to have nicknames as columns
pivoted_activity_data = pd.pivot_table(filtered_activity_data, values='ACTIVITY', 
                               index=['raw','hemisphere','marker'],columns=['nickname'], aggfunc=np.sum)

#Restarting index to activate'hemisphere','marker'and 'rater' as columns
pivoted_activity_data = pivoted_activity_data.reset_index()

In [22]:
# Changing directory to the main path
os.chdir(main_directory)

In [23]:
#combining dataframes to create the "cell_count_output.csv" file
allen_and_activity = pd.merge(brain_allen_map,pivoted_activity_data,on='raw',how='right')
cell_count_output = pd.merge(allen_and_activity,pivoted_volumes_data,on='raw',how='left')
cell_count_output.to_csv('cell_count_output.csv', index=False)

In [24]:
#Ending timming process
end = time.time()

# Printing how many files were processed and how much time the process took
files_processed = number_activity_files = number_files+ number_volume_files + number_volintissue_files
print(files_processed,' Files processed ')
print(' ')
print(' Execution time:', round((end - start),2), 'seconds') 

67  Files processed 
 
 Execution time: 3.13 seconds
